In [90]:
import pandas as pd
import numpy as np
import matplotlib as plt
import os
%matplotlib inline
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

from sklearn.preprocessing import OneHotEncoder
# !pip install category-encoders
from category_encoders.cat_boost import CatBoostEncoder

from collections import Counter
from imblearn.over_sampling import SMOTE, ADASYN

from sklearn.preprocessing import MinMaxScaler, StandardScaler

from sklearn.metrics import roc_auc_score, f1_score, accuracy_score, recall_score, precision_score, confusion_matrix, fbeta_score
from catboost import CatBoostClassifier
import dill

import warnings
warnings.filterwarnings("ignore") 

%matplotlib inline

In [74]:
valid = pd.read_csv('feat_cup_val_alfa.csv')

In [3]:
valid

,client_id,dt,accountsalary_flag,country,profit_income_out_rur_amt_6m,clientoutflowstatus,curbal_usd_amt_cm_avg,stratsegfactor,srvpackage,clientsegment,...,transaction_category_house_repair_inc_cnt_2m,transaction_category_cash_percent_cnt_2m,transaction_category_hotels_sum_cnt_m2,transaction_category_sporting_goods_inc_amt_2m,transaction_category_airplane_tickets_percent_amt_2m,transaction_category_transport_percent_amt_2m,transaction_category_restaurants_percent_amt_2m,transaction_category_auto_services_percent_amt_2m,transaction_category_supermarket_sum_cnt_m2,transaction_category_auto_services_sum_cnt_m2
0,482105f32bb380492d9f9c8f24dbea91b04da052e9c4d6...,2021-06-30,1.0,RUS,22269.500000,ACTIVE,5794.0,Пакет услуг,Премиум,AFFLUENT,...,1.120614,0.071429,NaN,NaN,NaN,0.002245,0.000432,0.079708,2.0,3.0
1,cd5844341331299f8a8e4fd879608e62c1aea20b13311e...,2022-03-31,1.0,RUS,7417.166667,ACTIVE,366.0,Остатки на счетах,Технический,MAFFLUENT,...,0.553030,0.021898,NaN,5.005737,NaN,0.016586,0.007430,0.027129,145.0,6.0
2,cf2384e8114c1eab1f20eb2066fe904156afff09d4039d...,2021-07-31,1.0,RUS,15453.833333,ACTIVE,6022.0,Пакет услуг,Максимум+,AFFLUENT,...,0.640351,0.063158,1.0,NaN,0.000196,0.000020,NaN,NaN,33.0,NaN
3,b7b5ec383e97d4553ece4ca7afb7dd9a96675c3934689e...,2022-03-31,1.0,RUS,14705.666667,ACTIVE,695.0,Пакет услуг,А-Клуб,A-CLUB,...,0.912500,0.029650,NaN,NaN,NaN,0.012834,0.101106,0.002461,131.0,4.0
4,002b29c1d20a331e5aa6e5079cf60496cf0b356173ec11...,2022-05-31,0.0,RUS,649.000000,SLEEP,0.0,NaN,Класс,MASS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28914,24d5ba759f91aeeb0fa5245f272c855c0a98c75341b578...,2022-07-31,0.0,RUS,164.000000,N2BANK_POTENTIAL,0.0,Инвестиционные продукты,Технический,MAFFLUENT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28915,2115faff4ee1e10a59eaca857930bc241a88c1875911a1...,2022-04-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28916,5e2de0998a46274e305ebefcc8dda2be1a5424d62bb1ba...,2022-05-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28917,d42bb0b661a276366c3c3a3502e0eb96fae432532d9316...,2023-01-31,1.0,RUS,-137.333333,ACTIVE,58.0,Остатки на счетах,Технический,MASS,...,1.850000,0.013100,NaN,NaN,NaN,0.025167,0.004948,NaN,99.0,NaN


In [6]:
valid[valid['gender'].isna()].iloc[0].values

array(['e235f23fc80f99c09c392f87876ebc68a67cccfca84de68d06c998684815d5f9',
       '2021-05-31', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 

In [7]:
na_ind = valid[valid['gender'].isna()].index

## Определяем типы переменных

In [9]:
data = pd.read_csv('df_promejut.csv')

#отбираем строковые переменные (КАТЕГОРИАЛЬНЫЕ)
xxx = data.select_dtypes(include = 'object')
xxx.head()

'''
смотрим: client_id - не признак
dt - дата
остальные: категориальные.
Правда, среди них могут быть и бинарные признаки. Выделим их:
'''
trash = ['client_id']
dates = ['dt']
xxx_categoricals = list(set(xxx.columns) - set(trash) - set(dates))

binaries = []
categoricals = []
for fea in xxx_categoricals:
    l = len(data[fea].value_counts())
    if l == 2:
        binaries.append(fea)
    elif l <= 1:
        trash.append(fea)
    else: #больше 2 уникальных значений за исключением nan
        categoricals.append(fea)
    
#отбираем числовые переменные
xxx = data.select_dtypes(include = 'number')
xxx.head()

'''
И как следовало ожидать, остальные 284 - числовые
target - исключаем из рассмотрения
'''
xxx.drop('target', axis = 1, inplace = True)
trash.append('target')

continuous = []
for fea in xxx.columns:
    l = len(data[fea].value_counts())
    if l == 2:
        binaries.append(fea)
    elif l <= 1:
        trash.append(fea)
    elif l>2 and l<=10: #больше 2 уникальных значений, но не более 10 за исключением nan
        categoricals.append(fea)
    else:
        continuous.append(fea)

categoricals.remove('hdb_bki_total_currency')
categoricals.remove('hdb_bki_active_oth_cnt')
continuous.append('hdb_bki_total_currency')
continuous.append('hdb_bki_active_oth_cnt')

del data

# '''
# НА ВЫХОДЕ:

# binaries - бинарные признаки
# categoricals - категориальные признаки
# continuous - непрерывные признаки
# dates - даты
# trash - мусор
# '''

In [25]:
all_feats = list(set(continuous) | set(categoricals) | set(binaries))
all_cats = list(set(categoricals) | set(binaries))

## Затестим отдельно, как наша модель работает на строках с na

In [75]:
df_wna = pd.read_csv('df_promejut.csv')
df_wna=df_wna[df_wna['gender'].isna()]
number_list = list(set(continuous).union(set(binaries)))

for i in number_list:
    if 'amount' in i or 'avg_' in i or 'profit_' in i or 'amt' in i or 'sum' in i or 'transaction' in i or 'cnt' in i or 'count' in i or 'diff' in i or 'active' in i or 'limit' in i or 'total_currency' in i or  'total_products' in i:
        df_wna[i]=df_wna[i].fillna(0)
        
for i in number_list:
    if 'life_time_days' in i:
        df_wna[i]=df_wna[i].fillna(0)

for i in number_list:
    if 'days' in i:
        df_wna[i]=df_wna[i].fillna(value=df[i].max()+1)
        
for i in number_list:
    if 'age' in i or 'total_inc' in i:
        df_wna[i]=df_wna[i].fillna(value=df[i].median())
        
for i in categoricals:
        df_wna[i]=df_wna[i].fillna(value='NA')
        
for i in all_feats:
        df_wna[i]=df_wna[i].fillna(value='NA')
        
        
assert df_wna.isna().sum().sum() == 0

## Основная пердобработка

In [76]:
df = pd.read_csv('df_train.csv')

In [77]:
number_list = list(set(continuous).union(set(binaries)))

for i in number_list:
    if 'amount' in i or 'avg_' in i or 'profit_' in i or 'amt' in i or 'sum' in i or 'transaction' in i or 'cnt' in i or 'count' in i or 'diff' in i or 'active' in i or 'limit' in i or 'total_currency' in i or  'total_products' in i:
        valid[i]=valid[i].fillna(0)
        
for i in number_list:
    if 'life_time_days' in i:
        valid[i]=valid[i].fillna(0)

for i in number_list:
    if 'days' in i:
        valid[i]=valid[i].fillna(value=df[i].max()+1)
        
for i in number_list:
    if 'age' in i or 'total_inc' in i:
        valid[i]=valid[i].fillna(value=df[i].median())
        
for i in categoricals:
        valid[i]=valid[i].fillna(value='NA')
        
for i in all_feats:
        valid[i]=valid[i].fillna(value='NA')
        
assert valid.isna().sum().sum() == 0

In [78]:
df['hdb_other_active_mean_psk'] = np.where(df['hdb_other_active_mean_psk'] <0, 0 ,df['hdb_other_active_mean_psk'] )
df['hdb_other_active_min_psk'] = np.where(df['hdb_other_active_min_psk'] <0, 0 ,df['hdb_other_active_min_psk'] )
df_wna['hdb_other_active_mean_psk'] = np.where(df_wna['hdb_other_active_mean_psk'] <0, 0 ,df_wna['hdb_other_active_mean_psk'] )
df_wna['hdb_other_active_min_psk'] = np.where(df_wna['hdb_other_active_min_psk'] <0, 0 ,df_wna['hdb_other_active_min_psk'] )
valid['hdb_other_active_mean_psk'] = np.where(valid['hdb_other_active_mean_psk'] <0, 0 ,valid['hdb_other_active_mean_psk'] )
valid['hdb_other_active_min_psk'] = np.where(valid['hdb_other_active_min_psk'] <0, 0 ,valid['hdb_other_active_min_psk'] )

In [31]:
features = df.drop(['target', 'dt', 'client_id'], axis=1).columns
#значения, которые нужно закодировать по CBE
large_uniq = []
for i in list(set(categoricals)): #.intersection(set(features))
    if len(df[i].unique()) > 20:
        large_uniq.append(i)

other_cat_feature = set(categoricals).intersection(set(features)) - set(large_uniq)
other_cat_feature

{'clientoutflowstatus',
 'clientsegment',
 'hdb_relend_client_credits_count_grade',
 'stratsegfactor'}

In [32]:
other_cat_feature = set(other_cat_feature).union(set(['gender']))
other_cat_feature

{'clientoutflowstatus',
 'clientsegment',
 'gender',
 'hdb_relend_client_credits_count_grade',
 'stratsegfactor'}

In [79]:
for i in large_uniq:
        CBE_encoder = CatBoostEncoder(random_state=42)
        df[i] = CBE_encoder.fit_transform(df[i], df['target'] )
        valid[i] = CBE_encoder.transform(valid[i])
        df_wna[i] = CBE_encoder.transform(df_wna[i])

In [43]:
class MultiColumnOneHotEncoder:
    def __init__(self):
        self.categories = {}
    
    def fit(self, data):
        for column in data.columns:
            unique_values = set(data[column])
            self.categories[column] = {}
            for index, value in enumerate(unique_values):
                self.categories[column][value] = index
            
    def transform(self, data):
        encoded_data = pd.DataFrame()
        for column in data.columns:
            for category, index in self.categories[column].items():
                encoded_data[f'{column}_{category}'] = (data[column] == category).astype(int)
        return encoded_data
  

In [80]:
encoder = MultiColumnOneHotEncoder()
encoder.fit(df[list(other_cat_feature)])
df_ohe = encoder.transform(df[list(other_cat_feature)])
df_ohe=df_ohe.drop(['gender_Женский'],axis=1)
for i in df_ohe.columns:
    df[i]=df_ohe[i].values.copy()
df=df.drop(other_cat_feature, axis=1)

In [81]:
df_ohe_valid = encoder.transform(valid[list(other_cat_feature)])
df_ohe_valid=df_ohe_valid.drop(['gender_Женский'],axis=1)
for i in df_ohe_valid.columns:
    valid[i]=df_ohe_valid[i].values.copy()
valid=valid.drop(other_cat_feature, axis=1)

In [82]:
df_ohe_wna = encoder.transform(df_wna[list(other_cat_feature)])
df_ohe_wna=df_ohe_wna.drop(['gender_Женский'],axis=1)
for i in df_ohe_wna.columns:
    df_wna[i]=df_ohe_wna[i].values.copy()
df_wna=df_wna.drop(other_cat_feature, axis=1)

In [66]:
df_wna['target'].sum()/df_wna.shape[0]

0.011245803250438245

In [52]:
features_new = list((set(features) - set(other_cat_feature)).union( set(df_ohe.columns)))

In [67]:
train = pd.read_csv('df_train_ADASYN_UP.csv')

In [56]:
feat=['clientsegment_MAFFLUENT',
 'transaction_category_cash_percent_amt_2m',
 'days_to_last_transaction',
 'hdb_bki_total_pil_cnt',
 'curr_rur_amt_cm_avg',
 'clientoutflowstatus_ACTIVE',
 'clientsegment_A-CLUB',
 'accountsalary_flag',
 'life_time_days',
 'gender_Мужской',
 'amount_by_category_90d__summarur_amt__SUM__cashflowcategory_name__tovary_i_uslugi',
 'amount_by_category_90d__summarur_amt__SUM__cashflowcategory_name__fastfud',
 'hdb_bki_total_max_limit',
 'hdb_bki_total_pil_max_limit',
 'sal_rur_amt_curr_v2',
 'transaction_category_supermarket_percent_cnt_2m',
 'amount_by_category_90d__summarur_amt__SUM__cashflowcategory_name__sportivnaja_odezhda',
 'total_inc',
 'transaction_category_pharmacy_percent_cnt_2m',
 'transaction_category_supermarket_sum_amt_m2',
 'curr_rur_amt_curr_v2',
 'curbal_usd_amt_cm_avg',
 'hdb_bki_total_cc_last_days',
 'hdb_bki_last_product_days',
 'stratsegfactor_Пакет услуг',
 'total_rur_amt_curr_v2',
 'avg_amount_daily_transactions_90d',
 'hdb_bki_total_active_products',
 'transaction_category_fastfood_percent_cnt_2m',
 'hdb_bki_total_pil_last_days',
 'transaction_category_supermarket_sum_cnt_m2',
 'stratsegfactor_Остатки на счетах',
 'profit_income_out_rur_amt_6m',
 'age',
 'hdb_other_active_mean_psk',
 'transaction_category_supermarket_inc_cnt_2m',
 'by_category__amount__SUM__eoperation_type_name__perevod_po_nomeru_telefona',
 'profit_income_out_rur_amt_12m',
 'clientoutflowstatus_POTENTIAL',
 'hdb_bki_total_cc_max_limit',
 'hdb_bki_total_cnt',
 'transaction_category_other_retail_purchase_percent_cnt_2m',
 'avg_cnt_daily_transactions_30d',
 'transaction_category_other_retail_purchase_sum_amt_m2',
 'transaction_category_pharmacy_inc_amt_2m',
 'amount_by_category_90d__summarur_amt__SUM__cashflowcategory_name__razvlechenija',
 'avg_credit_turn_rur',
 'dda_rur_amt_cm_avg',
 'by_category__amount__SUM__eoperation_type_name__vneshnij_perevod_rur',
 'amount_by_category_90d__summarur_amt__SUM__cashflowcategory_name__odezhda',
 'transaction_category_money_send_percent_amt_2m',
 'profit_income_out_rur_amt_3m',
 'diff_cur_cr_db_turn',
 'avg_cur_cr_turn']

In [83]:
for i in feat:
    sc = MinMaxScaler()
    train[i] = sc.fit_transform(np.expand_dims(train[i].to_numpy(), axis=1))
    valid[i] = sc.transform(np.expand_dims(valid[i].to_numpy(), axis=1))
    df_wna[i] = sc.transform(np.expand_dims(df_wna[i].to_numpy(), axis=1))

In [85]:
with open('NOT_USE_2_Voting_LGBM_CAT_XGBM.dat', 'rb') as f:
    model = dill.load(f)

[22:04:46] WARNING: ../src/learner.cc:553: 
  If you are loading a serialized model (like pickle in Python, RDS in R) generated by
  older XGBoost, please export the model by calling `Booster.save_model` from that version
  first, then load it back in current version. See:

    https://xgboost.readthedocs.io/en/latest/tutorials/saving_model.html

  for more details about differences between saving model and serializing.

[22:04:46] WARNING: ../src/learner.cc:553: 
  If you are loading a serialized model (like pickle in Python, RDS in R) generated by
  older XGBoost, please export the model by calling `Booster.save_model` from that version
  first, then load it back in current version. See:

    https://xgboost.readthedocs.io/en/latest/tutorials/saving_model.html

  for more details about differences between saving model and serializing.



In [86]:
model

VotingClassifier(estimators=[('lgbm',
                              LGBMClassifier(learning_rate=0.07, max_depth=8,
                                             min_child_samples=55,
                                             num_leaves=45, random_state=42,
                                             silent='warn', subsample=0.23)),
                             ('xgbc',
                              XGBClassifier(base_score=0.5, booster='gbtree',
                                            callbacks=None, colsample_bylevel=1,
                                            colsample_bynode=1,
                                            colsample_bytree=1,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_m...
                                            interaction_constraints='',
                                            learning_rate=0.15, max_bin=256,
                                            max_cat_threshold=64,
                                            max_cat_to_onehot=4,
                                            max_delta_step=0, max_depth=5,
                                            max_leaves=0, min_child_weight=1,
                                            missing=nan,
                                            monotone_constraints='()',
                                            n_estimators=100, n_jobs=0,
                                            num_parallel_tree=1,
                                            predictor='auto', random_state=42, ...)),
                             ('cat',
                              <catboost.core.CatBoostClassifier object at 0x7fe583f78a60>)],
                 voting='soft')

In [91]:
pred_proba = model.predict_proba(train[feat])[:,1]
print('gini: ', round(2*roc_auc_score(train['target'], pred_proba)-1,3))
pred = model.predict(train[feat])
print('f1_score: ', round(f1_score(train['target'], pred),3))
print('f2_score: ', round(fbeta_score(train['target'], pred, beta=2),3))
print('accuracy_score: ', round(accuracy_score(train['target'], pred),3))
print('recall_score: ', round(recall_score(train['target'], pred),3))
print('precision_score: ', round(precision_score(train['target'], pred),3))

gini:  0.869
f1_score:  0.848
f2_score:  0.845
accuracy_score:  0.849
recall_score:  0.842
precision_score:  0.854


In [92]:
pred_proba = model.predict_proba(df_wna[feat])[:,1]
print('gini: ', round(2*roc_auc_score(df_wna['target'], pred_proba)-1,3))
pred = model.predict(df_wna[feat])
print('f1_score: ', round(f1_score(df_wna['target'], pred),3))
print('f2_score: ', round(fbeta_score(df_wna['target'], pred, beta=2),3))
print('accuracy_score: ', round(accuracy_score(df_wna['target'], pred),3))
print('recall_score: ', round(recall_score(df_wna['target'], pred),3))
print('precision_score: ', round(precision_score(df_wna['target'], pred),3))

gini:  -0.011
f1_score:  0.037
f2_score:  0.026
accuracy_score:  0.988
recall_score:  0.021
precision_score:  0.163


In [93]:
confusion_matrix(df_wna['target'], pred)

array([[66475,    82],
       [  741,    16]])

In [102]:
model.predict_proba(df_wna[feat])[:,1].max()

0.6653211382505072

Все отлично - модель практически не ошибается (но в теории можно руками для таких наблюдений прописать скор = 0)

In [104]:
df_2 = pd.concat([train,df_wna])

pred_proba = model.predict_proba(df_2[feat])[:,1]
print('gini: ', round(2*roc_auc_score(df_2['target'], pred_proba)-1,3))
pred = model.predict(df_2[feat])
print('f1_score: ', round(f1_score(df_2['target'], pred),3))
print('f2_score: ', round(fbeta_score(df_2['target'], pred, beta=2),3))
print('accuracy_score: ', round(accuracy_score(df_2['target'], pred),3))
print('recall_score: ', round(recall_score(df_2['target'], pred),3))
print('precision_score: ', round(precision_score(df_2['target'], pred),3))

gini:  0.846
f1_score:  0.845
f2_score:  0.841
accuracy_score:  0.878
recall_score:  0.838
precision_score:  0.853


# Скорим валидационную выборку

In [107]:
pred_proba = model.predict_proba(valid[feat])[:,1]
pred = model.predict(valid[feat])

In [96]:
valid

,client_id,dt,accountsalary_flag,country,profit_income_out_rur_amt_6m,curbal_usd_amt_cm_avg,srvpackage,age,addrref,life_time_days,...,clientoutflowstatus_PREOUTFLOW,clientoutflowstatus_ACTIVE,gender_Мужской,stratsegfactor_nan,stratsegfactor_Предиктивный доход,stratsegfactor_Реальная зарплата клиента,stratsegfactor_Пакет услуг,stratsegfactor_Зарплата из кредитной анкеты,stratsegfactor_Инвестиционные продукты,stratsegfactor_Остатки на счетах
0,482105f32bb380492d9f9c8f24dbea91b04da052e9c4d6...,2021-06-30,1.0,0.064897,22269.500000,5794.0,0.285396,73.0,0.083043,6512.0,...,0,1.0,1.0,0,0,0,1.0,0,0,0.0
1,cd5844341331299f8a8e4fd879608e62c1aea20b13311e...,2022-03-31,1.0,0.064897,7417.166667,366.0,0.060562,43.0,0.083043,5102.0,...,0,1.0,1.0,0,0,0,0.0,0,0,1.0
2,cf2384e8114c1eab1f20eb2066fe904156afff09d4039d...,2021-07-31,1.0,0.064897,15453.833333,6022.0,0.350860,59.0,0.083043,5303.0,...,0,1.0,0.0,0,0,0,1.0,0,0,0.0
3,b7b5ec383e97d4553ece4ca7afb7dd9a96675c3934689e...,2022-03-31,1.0,0.064897,14705.666667,695.0,0.969332,58.0,0.067978,5359.0,...,0,1.0,1.0,0,0,0,1.0,0,0,0.0
4,002b29c1d20a331e5aa6e5079cf60496cf0b356173ec11...,2022-05-31,0.0,0.064897,649.000000,0.0,0.019740,82.0,0.057469,5313.0,...,0,0.0,1.0,0,0,0,0.0,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28914,24d5ba759f91aeeb0fa5245f272c855c0a98c75341b578...,2022-07-31,0.0,0.064897,164.000000,0.0,0.060562,59.0,0.083043,132.0,...,0,0.0,1.0,0,0,0,0.0,0,1,0.0
28915,2115faff4ee1e10a59eaca857930bc241a88c1875911a1...,2022-04-30,0.0,0.064675,0.000000,0.0,0.064675,39.0,0.064675,0.0,...,0,0.0,0.0,0,0,0,0.0,0,0,0.0
28916,5e2de0998a46274e305ebefcc8dda2be1a5424d62bb1ba...,2022-05-31,0.0,0.064675,0.000000,0.0,0.064675,39.0,0.064675,0.0,...,0,0.0,0.0,0,0,0,0.0,0,0,0.0
28917,d42bb0b661a276366c3c3a3502e0eb96fae432532d9316...,2023-01-31,1.0,0.064897,-137.333333,58.0,0.060562,28.0,0.068532,111.0,...,0,1.0,1.0,0,0,0,0.0,0,0,1.0


In [100]:
valid[['client_id', 'dt']+feat].to_csv('to_score.csv',index=False)

In [116]:
valid['score']=pred_proba
valid['predict']=pred

In [117]:
df_preds = valid[['client_id', 'dt','predict','score']]

In [118]:
df_preds.head()

,client_id,dt,predict,score
0,482105f32bb380492d9f9c8f24dbea91b04da052e9c4d6...,2021-06-30,1.0,0.762952
1,cd5844341331299f8a8e4fd879608e62c1aea20b13311e...,2022-03-31,1.0,0.865791
2,cf2384e8114c1eab1f20eb2066fe904156afff09d4039d...,2021-07-31,1.0,0.877971
3,b7b5ec383e97d4553ece4ca7afb7dd9a96675c3934689e...,2022-03-31,1.0,0.989112
4,002b29c1d20a331e5aa6e5079cf60496cf0b356173ec11...,2022-05-31,0.0,0.168768


In [120]:
score_file = pd.read_csv('score_file.csv')
score_file.drop('score',axis=1, inplace=True)

In [129]:
res = score_file.merge(df_preds[['client_id','score']], on='client_id')

In [140]:
res.head()

,client_id,score
0,3fbd8c4b629545972d6bcba74f367c10cabc0f2d2715f5...,0.263296
1,1ee25fb2b3b07d2f165d15bd9b95f0e6d8cfccf586468d...,0.529391
2,aea24d61bb2f0eeb0ff4f897ce4037f58eff29acbd4e36...,0.534691
3,1318bbf375035082f93aebf102249b110abda93d9970a1...,0.444462
4,6627d4565339a64bba25275af2eb8f3c1bcf3008ce3948...,0.765850


In [141]:
res.to_csv('score.csv',index=False)

In [142]:
res

,client_id,score
0,3fbd8c4b629545972d6bcba74f367c10cabc0f2d2715f5...,0.263296
1,1ee25fb2b3b07d2f165d15bd9b95f0e6d8cfccf586468d...,0.529391
2,aea24d61bb2f0eeb0ff4f897ce4037f58eff29acbd4e36...,0.534691
3,1318bbf375035082f93aebf102249b110abda93d9970a1...,0.444462
4,6627d4565339a64bba25275af2eb8f3c1bcf3008ce3948...,0.765850
...,...,...
28914,ed601bb0a2c253b8ec6e422baa7dc43579de235c7ec728...,0.664991
28915,da498a7086d793cba6deab7a2e3b70a1e2ed008ca37183...,0.550027
28916,5079d852b78fe378fd08cb3d4168c2c6eee42177c3acb3...,0.402641
28917,8ecc107ebc5c58c781828dfa5e555ac82a00c757bd5fa7...,0.236781
